In [ ]:
from IPython.core.display import HTML
with open('../../Python/style.css') as f:
    css = f.read()
HTML(css)

# Sudoku

The sudoku we want to solve is shown below:
    <img src="sudoku.png">
It can be represented as a list of lists.

In [ ]:
Sudoku = [ ["*",  3 ,  9 , "*", "*", "*", "*", "*",  7 ], 
           ["*", "*", "*",  7 , "*", "*",  4 ,  9 ,  2 ],
           ["*", "*", "*", "*",  6 ,  5 , "*",  8 ,  3 ],
           ["*", "*", "*",  6 , "*",  3 ,  2 ,  7 , "*"],
           ["*", "*", "*", "*",  4 , "*",  8 , "*", "*"],
           [ 5 ,  6 , "*", "*", "*", "*", "*", "*", "*"],
           ["*", "*",  5 ,  2 , "*",  9 , "*", "*",  1 ],
           ["*",  2 ,  1 , "*", "*", "*", "*",  4 , "*"],
           [ 7 , "*", "*", "*", "*", "*",  5 , "*", "*"]
         ]

The function `allDifferent` takes a list `L` as its argument.  This list contains both variables and numbers.
It returns a set of formulas expressing that all variables from `L` take different values.

In [ ]:
def allDifferent(Variables):
    return { f'{x} != {y}' for x in Variables 
                           for y in Variables
                           if  x < y
           }

In [ ]:
def constraints_from_puzzle(Puzzle):
    return { f'V{row+1}{col+1} == {Puzzle[row][col]}' for row in range(9)
                                                      for col in range(9)
                                                      if  Puzzle[row][col] != '*'
           }

In [ ]:
constraints_from_puzzle(Sudoku)

The function `sudoku_csp` returns a CSP that encodes the given sudoku as a CSP.

In [ ]:
def sudoku_csp(Puzzle): 
    L           = range(1, 9+1)
    Variables   = { f'V{row}{col}' for row in L
                                   for col in L
                  }
    Values      = set(L)
    Constraints = constraints_from_puzzle(Puzzle)
    # all entries in a row are unique
    for row in L:
        Constraints |= allDifferent({ f'V{row}{col}' for col in L })
    # all entries in a column are unique
    for col in L:
        Constraints |= allDifferent({ f'V{row}{col}' for row in L })
    # all entries in a square are unique    
    for r in range(3):
        for c in range(3):
            S = { f'V{r*3+row}{c*3+col}' for row in range(1, 3+1)
                                         for col in range(1, 3+1) 
                }
            Constraints |= allDifferent(S)
    return Variables, Values, Constraints

In [ ]:
sudoku_csp(Sudoku)

Given a sudoku `Puzzle` and a `Solution` for this puzzle, the function `find_alternative` computes a CSP
that is solvable iff the puzzle has a second solution that is different from `Solution`.

In [ ]:
def find_alternative(csp, Solution):
    Variables, Values, Constraints = csp
    formula = ' or '.join({ f'{var} != {Solution[var]}' for var in Variables })
    return Variables, Values, Constraints | { formula }

## Graphical Representation

In [ ]:
import ipycanvas as cnv

The function `show_solution` prints the solution.

In [ ]:
def show_sudoku(Solution, Puzzle):
    length = 9 * 4 + 5
    print("=" * length)
    for row in range(9):
        s = "|| "
        for col in range(9): 
            entry = Puzzle[row][col]
            if (col + 1) % 3 == 0:
                sep = ' || '
            else:
                sep = ' | '
            if entry != '*':
                s += str(entry) + sep
            elif f'V{row+1}{col+1}' in Solution:
                s += str(Solution[f'V{row+1}{col+1}']) + sep
            else:
                s += "*" + sep
        print(s)
        if (row + 1) % 3 == 0:
            print("=" * length)
        else:
            print("-" * length)

The second version of show solution draws the solution on a canvas.

In [ ]:
def show_solution(Solution, Sudoku):
    size = 40
    canvas = cnv.Canvas(size=(size * 9, size * 9))
    canvas.font = '20px sans-serif'
    canvas.text_align    = 'center'
    canvas.text_baseline = 'middle'
    for row in range(9):
        for col in range(9):
            x = col * size
            y = row * size
            canvas.line_width = 1.0
            canvas.stroke_rect(x, y, size, size)
            entry = Sudoku[row][col]
            if entry == '*':
                key = f'V{row+1}{col+1}'
                symbol = str(Solution.get(key, '?'))
                canvas.fill_style = 'blue'
            else:
                symbol = str(entry)
                canvas.fill_style = 'black'
            x += size // 2
            y += size // 2
            canvas.fill_text(symbol, x, y)
    canvas.line_width = 3.0
    for row in range(3):
        for col in range(3):
            x = 3 * col * size
            y = 3 * row * size
            canvas.stroke_rect(x, y, 3 * size, 3 * size)
    canvas.stroke_style = 'black'
    canvas.line_width = 6.0
    canvas.stroke_rect(0, 0, 9 * size, 9 * size) 
    display(canvas)

In [ ]:
show_solution({}, Sudoku)